## Using NN Module to create ANN

## Here is the plan

    1.First load dataset
    2.Basic preprocess
    3.Making ANN
        i.first layer 8 nodes and relu
        ii.second layer 3 nodes and relu
        ii.third layer 1 nodes and sigmoid
    4.Summary of Ann
    5.Accuracy

In [19]:
# import libraries

import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torchinfo import summary

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [20]:
df = pd.read_csv('data/Telco-Customer-Churn.csv')

In [21]:
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [22]:
df.drop(columns=['customerID','MultipleLines','InternetService','OnlineSecurity','OnlineBackup','DeviceProtection','TechSupport','StreamingTV','StreamingMovies','Contract','PaymentMethod'],inplace=True)


In [23]:
df.head()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,PaperlessBilling,MonthlyCharges,TotalCharges,Churn
0,Female,0,Yes,No,1,No,Yes,29.85,29.85,No
1,Male,0,No,No,34,Yes,No,56.95,1889.5,No
2,Male,0,No,No,2,Yes,Yes,53.85,108.15,Yes
3,Male,0,No,No,45,No,No,42.30,1840.75,No
4,Female,0,No,No,2,Yes,Yes,70.70,151.65,Yes


In [24]:
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'],errors='coerce')
df['TotalCharges'].fillna(df['TotalCharges'].median(),inplace=True)

/tmp/ipykernel_27300/931749561.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['TotalCharges'].fillna(df['TotalCharges'].median(),inplace=True)


In [25]:
cat_cols= ['Partner','Dependents','PhoneService','PaperlessBilling','Churn']
num_cols= ['tenure','MonthlyCharges','TotalCharges']

In [26]:
for col in cat_cols:
    df[col] = df[col].map({'Yes':1,'No':0})
    df[col] = df[col].fillna(df[col].mode([0]))
df['gender'] = df['gender'].map({'Male':1,'Female':0})
df['gender'] = df['gender'].fillna(df['gender'].mode([0]))

In [27]:
for col in df.select_dtypes(include=['object']).columns:
    df[col] = pd.to_numeric(df[col], errors='coerce')
    df[col] = df[col].fillna(df[col].mode([0]))

In [28]:
x= df.drop('Churn',axis=1)
y= df['Churn']

In [29]:
x_train,x_test,y_train,y_test = train_test_split(x,y,random_state=42,test_size=0.25)

In [30]:
scaler = StandardScaler()

x_train[num_cols] = scaler.fit_transform(x_train[num_cols])
x_test[num_cols] = scaler.transform(x_test[num_cols])

In [31]:
x_train.isnull().sum()

gender              0
SeniorCitizen       0
Partner             0
Dependents          0
tenure              0
PhoneService        0
PaperlessBilling    0
MonthlyCharges      0
TotalCharges        0
dtype: int64

In [33]:
torch_x_train = torch.tensor(x_train.values,dtype=torch.float32)
torch_y_train = torch.tensor(y_train.values,dtype=torch.float32)
torch_x_test = torch.tensor(x_test.values,dtype=torch.float32)
torch_y_test = torch.tensor(y_test.values,dtype=torch.float32)

## Now ANN Model

In [ ]:
class Model(nn.Module):
    def __init__(self,num_features):
        super().__init__()
        self.network = nn.Sequential(
            nn.Linear(num_features,8),
            nn.ReLU(),
            nn.Linear(8,3),
            nn.ReLU(),
            nn.Linear(3,1),
            nn.Sigmoid(),
        )

    def forward(self,x):
        out = self.network(x)
        return out

In [58]:
model = Model(torch_x_train.shape[1])

In [68]:
loss_fun = nn.BCELoss()

In [70]:
optimizer = torch.optim.SGD(model.parameters(),lr=0.001)

In [ ]:
epochs = 50

In [73]:
for epoch in range(epochs):
    # forward pass
    y_pred = model(torch_x_train)
    
    # compute loss
    loss = model.loss_fun(y_pred.squeeze(),torch_y_train)
    print(f'Epoch {epoch+1}/{epochs}, Loss: {loss.item():.4f}')
     # clear gradients
    optimizer.zero_grad()
    # backward pass
    loss.backward()
    # update weights
    optimizer.step()
   

Epoch 1/50, Loss: 0.7079
Epoch 2/50, Loss: 0.7078
Epoch 3/50, Loss: 0.7077
Epoch 4/50, Loss: 0.7076
Epoch 5/50, Loss: 0.7076
Epoch 6/50, Loss: 0.7075
Epoch 7/50, Loss: 0.7074
Epoch 8/50, Loss: 0.7073
Epoch 9/50, Loss: 0.7073
Epoch 10/50, Loss: 0.7072
Epoch 11/50, Loss: 0.7071
Epoch 12/50, Loss: 0.7071
Epoch 13/50, Loss: 0.7070
Epoch 14/50, Loss: 0.7069
Epoch 15/50, Loss: 0.7068
Epoch 16/50, Loss: 0.7068
Epoch 17/50, Loss: 0.7067
Epoch 18/50, Loss: 0.7066
Epoch 19/50, Loss: 0.7065
Epoch 20/50, Loss: 0.7065
Epoch 21/50, Loss: 0.7064
Epoch 22/50, Loss: 0.7063
Epoch 23/50, Loss: 0.7063
Epoch 24/50, Loss: 0.7062
Epoch 25/50, Loss: 0.7061
Epoch 26/50, Loss: 0.7060
Epoch 27/50, Loss: 0.7060
Epoch 28/50, Loss: 0.7059
Epoch 29/50, Loss: 0.7058
Epoch 30/50, Loss: 0.7057
Epoch 31/50, Loss: 0.7057
Epoch 32/50, Loss: 0.7056
Epoch 33/50, Loss: 0.7055
Epoch 34/50, Loss: 0.7055
Epoch 35/50, Loss: 0.7054
Epoch 36/50, Loss: 0.7053
Epoch 37/50, Loss: 0.7052
Epoch 38/50, Loss: 0.7052
Epoch 39/50, Loss: 0.

In [74]:
summary(model,input_size=(torch_x_train.shape))

Layer (type:depth-idx)                   Output Shape              Param #
Model                                    [5282, 1]                 --
├─Sequential: 1-1                        [5282, 1]                 --
│    └─Linear: 2-1                       [5282, 8]                 80
│    └─ReLU: 2-2                         [5282, 8]                 --
│    └─Linear: 2-3                       [5282, 3]                 27
│    └─ReLU: 2-4                         [5282, 3]                 --
│    └─Linear: 2-5                       [5282, 1]                 4
│    └─Sigmoid: 2-6                      [5282, 1]                 --
Total params: 111
Trainable params: 111
Non-trainable params: 0
Total mult-adds (Units.MEGABYTES): 0.59
Input size (MB): 0.19
Forward/backward pass size (MB): 0.51
Params size (MB): 0.00
Estimated Total Size (MB): 0.70

## Evaluation

In [75]:
with torch.no_grad():
    y_pred = model(torch_x_test)
    y_pred = (y_pred >= 0.5).float()
    accuracy = (y_pred == torch_y_test).float().mean()
    print('Accuracy : ',accuracy.item())

Accuracy :  0.3105863928794861
